## Importing Libraries

In [1]:
import tensorflow_datasets as tfds
import numpy as np
import tensorflow as tf

## Data Preprocessing

In [2]:
mnist_dataset,mnist_info = tfds.load(name='mnist',with_info=True,as_supervised=True)

In [3]:
mnist_train , mnist_test = mnist_dataset['train'] , mnist_dataset['test']

num_valid_samples = 0.1 * mnist_info.splits['train'].num_examples
num_valid_samples = tf.cast(num_valid_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)


def scale(image,label):
    image = tf.cast(image , tf.float32)
    image /= 255.  #the images are grayscaled,hence there are only 255 different shades
    return image,label

scaled_train_and_valid_data = mnist_train.map(scale)
scaled_test_data = mnist_test.map(scale)

In [4]:
Buffer_size = 10000

shuffled_train_and_valid_data = scaled_train_and_valid_data.shuffle(Buffer_size)

validation_data = shuffled_train_and_valid_data.take(num_valid_samples)
train_data = shuffled_train_and_valid_data.skip(num_valid_samples)


In [5]:
batch_size=100
train_data = train_data.batch(batch_size)
validation_data = validation_data.batch(num_valid_samples) 
scaled_test_data = scaled_test_data.batch(num_test_samples)             

validation_inputs,validation_targets = next(iter(validation_data))

## Model using TF

In [27]:
input_size = 784 #since the size of 1 image is 28 x 28 pixels
output_size = 10 #since there are 10 digits
hiddenlayer_size = 200

model = tf.keras.Sequential([
                             tf.keras.layers.Flatten(input_shape = (28,28,1)),
                             tf.keras.layers.Dense(hiddenlayer_size,activation = 'relu'),
                             tf.keras.layers.Dense(hiddenlayer_size,activation = 'relu'),
                             tf.keras.layers.Dense(output_size,activation = 'softmax')      
                           ])

In [28]:
model.compile(optimizer = 'adam',loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
#choosing the optimization algo. and loss function

## Training the Model

In [29]:
num_epochs = 5

model.fit(train_data, epochs=num_epochs,validation_data = (validation_inputs,validation_targets),validation_steps=1, verbose = 2)

Epoch 1/5
540/540 - 2s - loss: 0.2717 - accuracy: 0.9211 - val_loss: 0.1272 - val_accuracy: 0.9627 - 2s/epoch - 4ms/step
Epoch 2/5
540/540 - 2s - loss: 0.1053 - accuracy: 0.9681 - val_loss: 0.0905 - val_accuracy: 0.9720 - 2s/epoch - 3ms/step
Epoch 3/5
540/540 - 2s - loss: 0.0687 - accuracy: 0.9787 - val_loss: 0.0765 - val_accuracy: 0.9768 - 2s/epoch - 3ms/step
Epoch 4/5
540/540 - 2s - loss: 0.0509 - accuracy: 0.9836 - val_loss: 0.0589 - val_accuracy: 0.9812 - 2s/epoch - 3ms/step
Epoch 5/5
540/540 - 2s - loss: 0.0388 - accuracy: 0.9874 - val_loss: 0.0529 - val_accuracy: 0.9832 - 2s/epoch - 3ms/step


In [31]:
test_loss,test_accuracy = model.evaluate(scaled_test_data)

1/1 [==============================] - 0s 473ms/step - loss: 0.0838 - accuracy: 0.9734


In [35]:
print(test_loss,test_accuracy)

0.0838194414973259 0.9733999967575073
